# Topic Modelling
<img src='https://miro.medium.com/v2/resize:fit:620/1*xi7Zdo4jUzaIwXTV58roYA.png'>

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings 
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv("articles.csv", encoding = 'latin1')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sabri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sabri\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Konu Modelleme yani topic modelling , metin belgelerindeki gizli konuları ortaya çıkarmak için kullanılan bir Doğal Dil İşleme tekniğidir. Bir metin belgesinin içeriği ile konu arasındaki ilişkileri bulmak için metin belgelerinin konularını belirlemeye yardımcı olur.

# Herhangi bir metin belgesinin konularını belirlemek için, içerik ve konular arasındaki ilişkileri tanımlamak üzere kelimelerin sıklığını analiz edebilen algoritmalar kullanmamız gerekir. Bu sorunu çözmek için metinsel verilere ihtiyacımız var.


In [18]:
df.head()

,Article,Title
0,data analysis process inspecting exploring dat...,Best Books to Learn Data Analysis
1,performance machine learning algorithm particu...,Assumptions of Machine Learning Algorithms
2,must seen news divided category go news websit...,News Classification with Machine Learning
3,two class classification problem problem binar...,Multiclass Classification Algorithms in Machin...
4,multinomial naive bayes one variant naive baye...,Multinomial Naive Bayes in Machine Learning


In [19]:
df.shape

(34, 2)

In [14]:
df.iloc[1,:][1],"***",df.iloc[1,:][0]

('Assumptions of Machine Learning Algorithms',
 '***',
 'The performance of a machine learning algorithm on a particular dataset often depends on whether the features of the dataset satisfies the assumptions of that machine learning algorithm. Not all machine learning algorithms have assumptions that differentiate them from each other. So, in this article, I will take you through the assumptions of machine learning algorithms.')

#### Dartamızda konu ve başlıklar var er halde buna göre dağoılımları yapacağız ve metin işleme ön adımlarını gerçekleştireceğiz 

In [15]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text


In [16]:
df['Article'] = df['Article'].apply(preprocess)

In [17]:
vectorizer = TfidfVectorizer(ngram_range=(2,2),stop_words='english')
x = vectorizer.fit_transform(df['Article'].values)

## Latent Dirichlet Allocation (LDA), metinsel verilerden oluşan bir külliyatta altta yatan konuları ortaya çıkarmak için kullanılan üretken bir olasılıksal algoritmadır.

In [20]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(x)

topic_modelling = lda.transform(x)

topic_labels = np.argmax(topic_modelling, axis=1)
df['topic_labels'] = topic_labels

df.head()

,Article,Title,topic_labels
0,data analysis process inspecting exploring dat...,Best Books to Learn Data Analysis,1
1,performance machine learning algorithm particu...,Assumptions of Machine Learning Algorithms,2
2,must seen news divided category go news websit...,News Classification with Machine Learning,0
3,two class classification problem problem binar...,Multiclass Classification Algorithms in Machin...,0
4,multinomial naive bayes one variant naive baye...,Multinomial Naive Bayes in Machine Learning,4


In [21]:
df.topic_labels.unique()

array([1, 2, 0, 4, 3], dtype=int64)

# Konu Modelleme, metin belgelerindeki gizli konuları ortaya çıkarmak için kullanılan bir Doğal Dil İşleme tekniğidir. 